# vdom with React components

This branch of jupyterlab includes a forked version of @nteract/transform-vdom that enables vdom to render React component in addition to native HTML elements.

Let's start by creating a vdom object from scratch and rendering it using ipython's display function:

In [41]:
from IPython.display import HTML, display

display({
    'application/vdom.v1+json': {
        'tagName': 'Button', 
        'attributes': {}, 
        'children': ['click me'],
        'import': {
            'package': '@blueprintjs/core',
            'module': 'Button'
        }
    }
}, raw=True)

A few things to notice:

* We have included a new `import` key in our vdom object
* `import` allows the user to specify a React component to import and use in lieu of a native HTML element
* In the above example, we are specifying an npm package (@blueprintjs/core) and named import or _module_ from that package (Button)
* When an `import` key is provided, transform-vdom dynamically import the specified module from npm (using dev.jspm.io) and replace the vdom component's `tagName` property with the imported React component.

TLDR;

The above output is a React component that was dynamically imported from npm.

Next, let's create a convenient function that will allow us to import React components and use them like we would any other vdom component. Let's call this function `import_component` and let's model it after vdom's own `create_component`:

In [42]:
from vdom import create_component, create_event_handler, VDOM

class ImportedVDOM(VDOM):
    __slots__ = ['tag_name', 'attributes', 'style', 'children', 'key', 'event_handlers', 'package', 'module', '_frozen']
    def __init__(
        self,
        tag_name,
        attributes=None,
        style=None,
        children=None,
        key=None,
        event_handlers=None,
        schema=None,
        package=None,
        module=None
    ):
        self.package = package
        self.module = module
        super().__init__(
            tag_name,
            attributes,
            style,
            children,
            key,
            event_handlers,
            schema
        )
        
    def to_dict(self):
        """Converts VDOM object to a dictionary that passes our schema
        """
        attributes = dict(self.attributes.items())
        if self.style:
            attributes.update({"style": dict(self.style.items())})
        vdom_dict = {'tagName': self.tag_name, 'attributes': attributes}
        if self.event_handlers:
            event_handlers = dict(self.event_handlers.items())
            for key, value in event_handlers.items():
                value = create_event_handler(key, value)
                event_handlers[key] = value
            vdom_dict['eventHandlers'] = event_handlers
        if self.key:
            vdom_dict['key'] = self.key
        vdom_dict['children'] = [c.to_dict() if isinstance(c, VDOM) else c for c in self.children]
        vdom_dict['import'] = {'package': self.package, 'module': self.module}
        return vdom_dict
    
def import_component(package, module=None):
    def _component(*children, **kwargs):
        if 'children' in kwargs:
            children = kwargs.pop('children')
        else:
            # Flatten children under specific circumstances
            # This supports the use case of div([a, b, c])
            # And allows users to skip the * operator
            if len(children) == 1 and isinstance(children[0], list):
                # We want children to be tuples and not lists, so
                # they can be immutable
                children = tuple(children[0])
        style = None
        event_handlers = None
        attributes = dict(**kwargs)
        if 'style' in kwargs:
            style = kwargs.pop('style')
        if 'attributes' in kwargs:
            attributes = kwargs['attributes']
        for key, value in attributes.items():
            if callable(value):
                attributes = attributes.copy()
                if event_handlers == None:
                    event_handlers = {key: attributes.pop(key)}
                else:
                    event_handlers[key] = attributes.pop(key)
        v = ImportedVDOM('div', attributes, style, children, None, event_handlers, package=package, module=module)
        return v

    return _component

Excellent, now we can simply:

In [43]:
Button = import_component('@blueprintjs/core', 'Button')

Button('test')

<div>test</div>

Ok, so we can import a Button component from the [blueprintjs](https://blueprintjs.com/docs/#core) package and render it. Yippee! What else can we do?

In [44]:
count = 1

def handle_click(event):
    global count
    count += 1
    counter.update(render_counter())

def render_counter():
    return Button(str(count), onClick=handle_click)

counter = display(render_counter(), display_id=True)

counter;

<div>5</div>

Oh wow, we can handle events like clicks using Python functions! 

In [45]:
Progress = import_component('@blueprintjs/core', 'ProgressBar')

Progress(value=0.5)

<div></div>

Ok, I progress bar is a little more exciting, but aren't there native HTML progress elements? Show me something exciting that I don't get without React...

In [46]:
CanvasDraw = import_component('react-canvas-draw')

CanvasDraw(width="100%")

<div width="100%"></div>

We don't hear too much about audio or music in Jupyter Notebooks (except from [Carol Willing](https://www.youtube.com/watch?v=DmfspNLML_k)), maybe because there aren't ipywidgets for it, so let's render a piano using [react-piano-component](https://github.com/lillydinhle/react-piano-component).

In [48]:
Piano = import_component('./ReactPianoComponent.js')

Piano()

<div></div>